In [1]:
import numpy as np
from sklearn import cluster
from osgeo import gdal, gdal_array 
import os

In [3]:
img_ds = gdal.Open('D:/work/kmeans/images/2024_01_30_cutted.tif')

In [4]:
num_rasters = img_ds.RasterCount
    
img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, num_rasters),
                   gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))

for b in range(num_rasters):
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    

In [5]:
img

array([[[2185., 2432., 3275.],
        [2390., 2587., 3365.],
        [2301., 2431., 3204.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       [[2223., 2486., 3260.],
        [2323., 2512., 3253.],
        [2271., 2403., 3214.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       [[2174., 2409., 3185.],
        [2185., 2441., 3198.],
        [2146., 2399., 3194.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       ...,

       [[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        ...,
        [2372., 2510., 3227.],
        [2313., 2478., 3193.],
        [2217., 2424., 3225.]],

       [[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        ...,
        [2129., 2389., 3205.],
        [2271., 2502., 3272.],
        [2220.,

In [6]:
img.shape

(424, 438, 3)

In [7]:
img_2 = img_ds.ReadAsArray()

In [8]:
img_2

array([[[2185., 2390., 2301., ...,    0.,    0.,    0.],
        [2223., 2323., 2271., ...,    0.,    0.,    0.],
        [2174., 2185., 2146., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ..., 2372., 2313., 2217.],
        [   0.,    0.,    0., ..., 2129., 2271., 2220.],
        [   0.,    0.,    0., ..., 2092., 2335., 2376.]],

       [[2432., 2587., 2431., ...,    0.,    0.,    0.],
        [2486., 2512., 2403., ...,    0.,    0.,    0.],
        [2409., 2441., 2399., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ..., 2510., 2478., 2424.],
        [   0.,    0.,    0., ..., 2389., 2502., 2511.],
        [   0.,    0.,    0., ..., 2409., 2514., 2542.]],

       [[3275., 3365., 3204., ...,    0.,    0.,    0.],
        [3260., 3253., 3214., ...,    0.,    0.,    0.],
        [3185., 3198., 3194., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ..., 3227., 3193., 3225.],
        [   0.,    0.,    0., ..., 3205., 327

In [9]:
img_2.shape

(3, 424, 438)

In [11]:
np.moveaxis(img_2, 0, -1)

array([[[2185., 2432., 3275.],
        [2390., 2587., 3365.],
        [2301., 2431., 3204.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       [[2223., 2486., 3260.],
        [2323., 2512., 3253.],
        [2271., 2403., 3214.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       [[2174., 2409., 3185.],
        [2185., 2441., 3198.],
        [2146., 2399., 3194.],
        ...,
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       ...,

       [[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        ...,
        [2372., 2510., 3227.],
        [2313., 2478., 3193.],
        [2217., 2424., 3225.]],

       [[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.],
        ...,
        [2129., 2389., 3205.],
        [2271., 2502., 3272.],
        [2220.,

In [12]:
img_ds.RasterXSize

438